In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [2]:
chromedriver = '/Users/fromh/Downloads/chromedriver'
driver = webdriver.Chrome(chromedriver)

In [3]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url + quote('넷플릭스추천')
driver.get(url)

In [4]:
# blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
lis = soup.select('li.bx._svp_item')
len(lis)

120

In [7]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, content, date

('넷플릭스 미드 추천',
 '추천 ☺️ 이 외에도 <김씨네 편의점> <루시퍼> <블랙미러> 등등 추천드릴만한 작품이 많긴 한데 일단 이 정도 추천드려봐요 ㅎㅎ 다음에 시간 날 때 추가 포스팅 하겠습니다. 즐넷플릭스 하세요 ! +) 넷플릭스 추천 포스팅...',
 '2021.06.02.')

In [8]:
title_list, content_list, date_list = [],[],[]
for i, li in enumerate(lis):
    try:
        adv = li.select_one('a.link_ad').get_text()
        if adv.strip() == '광고':
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

        #print(i)

In [9]:
df = pd.DataFrame({
    'title':title_list, 'date':date_list, 'content':content_list
})
df.head()

,title,date,content
0,넷플릭스 미드 추천,2021.06.02.,추천 ☺️ 이 외에도 <김씨네 편의점> <루시퍼> <블랙미러> 등등 추천드릴만한 작...
1,"넷플릭스 드라마 영화 추천 - 검블유, 클래식 외 8편",2021.07.25.,넷플릭스 영화 추천 - 영화 클래식 그간 지속적으로 넷플릭스 추천 포스팅을 썼는데 ...
2,넷플릭스 드라마 추천 8편! 시간 순삭!,2021.05.16.,그래서 넷플릭스 드라마 추천10편 가지고 왔습니다! 취향 비슷하신 분들이 많길 바라...
3,넷플릭스 드라마 추천! 안 보면 후회할 한국 드라마 BEST 10,2021.08.12.,액션·로맨스·시대물 다~ 있다넷플릭스 한국 드라마 추천 10 미생 출처 - tv...
4,아직도 못 봤다고? 넷플릭스 영국 드라마 추천작 Top 5,2021.04.01.,넷플릭스 영국 드라마 / 넷플릭스 영드 추천1. 오티스의 비밀 상담소 넷플릭스 영국...


In [10]:
df.to_csv('data/netflix_blog.txt')

In [11]:
driver.close()